#  Readme 

A simple notebook for monitoring changes in the GTFS. Developed for checking changes in routes start times between two dates.

The output is a DataFrame summarizing all changes (added/deleted elements) by route_id.

How to use this notebook?

1. set your credentials for using Splunk (Imports & settings/set your credentials for using Splunk) 

2. set dates to test (Get gtfs data/set dates)

3. set GTFS column to track (TrackGTFS/find_gtfs_changes)

4. optional - for monitoring changes in all_start_time column in specific time slot, use the set_deleted_between function (TrackGTFS/add deleted by hours cols)




# Imports & settings

## imports 

In [1]:
import pandas as pd
import numpy as np
import json

from IPython.display import  display

import os
import sys
sys.path.append('\\'.join(os.getcwd().split('\\')[:-2])) #add open_bus dir to path

from TrackGTFS import *
from splunk.SplunkUtils.SplunkUtils import *

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 40

## set your credentials for using Splunk

The most easy way is to put your credentials.json file in this notebook directory.

Learn more about SplunkUtils here: https://github.com/hasadna/open-bus/tree/master/splunk/SplunkUtils

In [2]:
with open('credentials.json', 'r') as fp:
    credentials = json.load(fp)

host, port, username, password = credentials.values()

# Get gtfs data

## set dates 

In [3]:
old_date, new_date =  "2019-11-30", "2020-01-11"

## set gtfs_old

In [4]:
query_kwargs_old = {'index': 'route_stats',
                   'date': old_date}

In [5]:
gtfs_old = read_splunk(splunk_query_builder(**query_kwargs_old),
                                host=host, port=port, username=username, password=password, time_limit=2)

start..

your query:
 search index=route_stats date=2019-11-30 |
 fields * |


connection succeed

query status: 100.0%   819251 scanned   2779 matched   2779 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 2779



In [6]:
gtfs_old.head(3)

,agency_id,agency_name,all_start_time,all_stop_code,all_stop_desc_city,all_stop_id,all_stop_latlon,all_stop_name,all_trip_id,date,date_hour,date_mday,date_minute,date_month,date_second,date_wday,date_year,date_zone,end_stop_city,end_stop_desc,end_stop_id,end_stop_lat,end_stop_lon,end_stop_name,end_time,end_zone,eventtype,gtfs_file_name,host,index,is_bidirectional,is_loop,linecount,max_headway,mean_headway,mean_trip_distance,mean_trip_duration,min_headway,num_stops,num_trip_ends,num_trip_starts,num_trips,num_zones,num_zones_missing,peak_end_time,peak_num_trips,peak_start_time,punct,route_alternative,route_direction,route_id,route_long_name,route_mkt,route_short_name,route_type,service_distance,service_duration,service_speed,source,sourcetype,splunk_server,splunk_server_group,start_stop_city,start_stop_desc,start_stop_id,start_stop_lat,start_stop_lon,start_stop_name,start_time,start_zone,xD7
0,3,אגד,23:59:00;23:59:00;23:59:00,21256;38500;11168;16179;19757;13907;...,תל אביב יפו;חבל יבנה;באר טוביה;קרית ...,37682;25399;25704;28327;29243;36386;...,"32.055898,34.779081;31.8220229999999...",ת.מרכזית ת''א ק.6/רציפים;צומת בני דר...,30595803_301119;30595804_301119;3059...,2019-11-30,23,30,59,november,0,saturday,2019,120,אילת,רחוב: עיר: אילת רציף: קומה:,12161,29.556961,34.952441,ת. מרכזית אילת/הורדה,28:46:21,NaN,NaN,2019-11-30T20-50-08_israel-public-tr...,MOT,route_stats,0,0,1,NaN,NaN,354747.0,4.789167,NaN,39,0,3,3,7,26,28:46:21,3,23:59:00,",,,,,._''_./-__<->.__/--,,,,,,,,,,::...",ז,1,7449,ת.מרכזית ת''א ק.6/רציפים-תל אביב יפו...,10394,394,3,1064241,14.367500,74.072803,/mnt/disks/splunk1/inputs/route_stat...,route_stats,splunk-2,NaN,תל אביב יפו,רחוב:תחנה מרכזית קומה 6 עיר: תל אבי...,37682,32.055898,34.779081,ת.מרכזית ת''א ק.6/רציפים,23:59:00,גוש דן,2455
1,25,אפיקים,23:59:00,33599;32800;38654;35348;36764;38299;...,פתח תקווה;פתח תקווה;פתח תקווה;פתח תק...,43301;16706;33962;23270;24296;25262;...,"32.097957,34.885102;32.0935349999999...",מסוף משה ארנס/רציפים;בר כוכבא/תל חי;...,35494287_301119,2019-11-30,23,30,59,november,0,saturday,2019,120,פתח תקווה,רחוב: עיר: פתח תקווה רציף: 1 קומה:,43328,32.097215,34.885440,מסוף משה ארנס/הורדה,24:51:43,גוש דן,NaN,2019-11-30T20-50-08_israel-public-tr...,MOT,route_stats,0,1,1,NaN,NaN,18075.0,0.878611,NaN,63,0,1,1,1,0,24:51:43,1,23:59:00,",,,,,__/-_<->__/-_-,,,,,,,,,,::,::,,...",1,3,1183,מסוף משה ארנס/רציפים-פתח תקווה<->מסו...,26099,99א,3,18075,0.878611,20.572242,/mnt/disks/splunk1/inputs/route_stat...,route_stats,splunk-2,NaN,פתח תקווה,רחוב: עיר: פתח תקווה רציף: 4 קומה:,43301,32.097957,34.885102,מסוף משה ארנס/רציפים,23:59:00,גוש דן,374
2,25,אפיקים,23:59:00,32298;32458;32457;32269;38315;38316;...,פתח תקווה;פתח תקווה;פתח תקווה;פתח תק...,16232;16384;16383;16205;25278;25279;...,"32.097013000000004,34.853018;32.0958...",חניון קרית אריה/החרש;אבשלום גיסין/מש...,32163899_301119,2019-11-30,23,30,59,november,0,saturday,2019,120,פתח תקווה,רחוב:דרך מנחם בגין 125 עיר: פתח תקוו...,16377,32.070170,34.913653,בית ספר בית יעקב/דרך מנחם בגין,24:34:54,גוש דן,NaN,2019-11-30T20-50-08_israel-public-tr...,MOT,route_stats,0,0,1,NaN,NaN,12474.0,0.598333,NaN,39,0,1,1,1,0,24:34:54,1,23:59:00,",,,,,__/-_<->___/__-_-,,,,,,,,,,::,:...",0,1,1175,חניון קרית אריה/החרש-פתח תקווה<->בית...,26098,98,3,12474,0.598333,20.847911,/mnt/disks/splunk1/inputs/route_stat...,route_stats,splunk-2,NaN,פתח תקווה,רחוב:החרש עיר: פתח תקווה רציף: קומה:,16232,32.097013,34.853018,חניון קרית אריה/החרש,23:59:00,גוש דן,356


In [7]:
gtfs_old.route_id.value_counts().head(3)

2132     1
17716    1
12409    1
Name: route_id, dtype: int64

## set gtfs_new 

In [8]:
query_kwargs_new = {'index': 'route_stats',
                   'date': new_date}

In [9]:
gtfs_new = read_splunk(splunk_query_builder(**query_kwargs_new),
                                host=host, port=port, username=username, password=password, time_limit=2)

start..

your query:
 search index=route_stats date=2020-01-11 |
 fields * |


connection succeed

query status: 100.0%   40943 scanned   2764 matched   2764 results

Done!

query succeed

read results succeed

job finished and canceled

finished! number of rows: 2764



In [10]:
gtfs_new.head(3)

,agency_id,agency_name,all_start_time,all_stop_code,all_stop_desc_city,all_stop_id,all_stop_latlon,all_stop_name,all_trip_id,cluster_id,cluster_name,cluster_sub_desc,date,date_hour,date_mday,date_minute,date_month,date_second,date_wday,date_year,date_zone,end_stop_city,end_stop_desc,end_stop_id,end_stop_lat,end_stop_lon,end_stop_name,end_time,end_zone,eventtype,host,index,is_bidirectional,is_loop,line_type,line_type_desc,linecount,max_headway,mean_headway,mean_trip_distance,mean_trip_duration,min_headway,num_stops,num_trip_ends,num_trip_starts,num_trips,num_zones,num_zones_missing,peak_end_time,peak_num_trips,peak_start_time,punct,route_alternative,route_direction,route_id,route_long_name,route_mkt,route_short_name,route_type,service_distance,service_duration,service_speed,source,source_files,sourcetype,splunk_server,splunk_server_group,start_stop_city,start_stop_desc,start_stop_id,start_stop_lat,start_stop_lon,start_stop_name,start_time,start_zone
0,3,אגד,23:59:00;23:59:00,20001;19757;12087;12394;10265,תל אביב יפו;בני שמעון;הערבה התיכונה;...,42872;29243;42415;43634;12161,"32.083157,34.796091;31.4423039999999...",ת. רכבת ת''א מרכז;תחנת התרעננות/פונד...,43119139_110120;43119140_110120,101,אילת עירוני ובינעירוני,NaN,2020-01-11,23,11,59,january,0,saturday,2020,120,אילת,רחוב: עיר: אילת רציף: קומה:,12161,29.556961,34.952441,ת. מרכזית אילת/הורדה,28:17:04,NaN,NaN,MOT,route_stats,0,0,3,בינעירוני,1,NaN,NaN,351493.0,4.301111,NaN,5,0,2,2,2,3,28:17:04,2,23:59:00,",,,,.__''_-__<->.__/--#,,,,#,,,,,,::...",#,1,5292,ת. רכבת ת''א מרכז-תל אביב יפו<->ת. מ...,10790,790,3,702986,8.602222,81.721441,/mnt/disks/splunk1/inputs/route_stat...,2020-01-11T20-50-06_ClusterToLine.zi...,route_stats,splunk-2,NaN,תל אביב יפו,רחוב: עיר: תל אביב יפו רציף: 10 קומה:,42872,32.083157,34.796091,ת. רכבת ת''א מרכז,23:59:00,גוש דן
1,3,אגד,23:59:00,20001;38500;11168;16179;19757;13907;...,תל אביב יפו;חבל יבנה;באר טוביה;קרית ...,42872;25399;25704;28327;29243;36386;...,"32.083157,34.796091;31.8220229999999...",ת. רכבת ת''א מרכז;צומת בני דרום;צומת...,43119155_110120,101,אילת עירוני ובינעירוני,NaN,2020-01-11,23,11,59,january,0,saturday,2020,120,אילת,רחוב: עיר: אילת רציף: קומה:,12161,29.556961,34.952441,ת. מרכזית אילת/הורדה,28:48:36,NaN,NaN,MOT,route_stats,0,0,3,בינעירוני,1,NaN,NaN,358176.0,4.826667,NaN,40,0,1,1,7,27,28:48:36,1,23:59:00,",,,,.__''_-__<->.__/--#,,,,#,,,,,,::...",#,1,26583,ת. רכבת ת''א מרכז-תל אביב יפו<->ת. מ...,15794,794,3,358176,4.826667,74.207735,/mnt/disks/splunk1/inputs/route_stat...,2020-01-11T20-50-06_ClusterToLine.zi...,route_stats,splunk-2,NaN,תל אביב יפו,רחוב: עיר: תל אביב יפו רציף: 10 קומה:,42872,32.083157,34.796091,ת. רכבת ת''א מרכז,23:59:00,גוש דן
2,25,אפיקים,23:59:00,33599;32800;38654;35348;36764;38299;...,פתח תקווה;פתח תקווה;פתח תקווה;פתח תק...,43301;16706;33962;23270;24296;25262;...,"32.097957,34.885102;32.0935349999999...",מסוף משה ארנס/רציפים;בר כוכבא/תל חי;...,35494287_110120,207,פתח תקוה-ראש העין,NaN,2020-01-11,23,11,59,january,0,saturday,2020,120,פתח תקווה,רחוב: עיר: פתח תקווה רציף: 1 קומה:,43328,32.097215,34.885440,מסוף משה ארנס/הורדה,24:51:43,גוש דן,NaN,MOT,route_stats,0,1,1,עירוני,1,NaN,NaN,18075.0,0.878611,NaN,63,0,1,1,1,0,24:51:43,1,23:59:00,",,,,__/-_<->__/-_-,,,,,,,,,,::,::,,,...",1,3,1183,מסוף משה ארנס/רציפים-פתח תקווה<->מסו...,26099,99א,3,18075,0.878611,20.572242,/mnt/disks/splunk1/inputs/route_stat...,2020-01-11T20-50-06_ClusterToLine.zi...,route_stats,splunk-2,NaN,פתח תקווה,רחוב: עיר: פתח תקווה רציף: 4 קומה:,43301,32.097957,34.885102,מסוף משה ארנס/רציפים,23:59:00,גוש דן


In [11]:
gtfs_new.route_id.value_counts().head(3)

2132     1
15667    1
19775    1
Name: route_id, dtype: int64

# TrackGTFS

##  find_gtfs_changes

In [12]:
col2track = 'all_start_time'

In [13]:
res = find_gtfs_changes(gtfs_old=gtfs_old, gtfs_new=gtfs_new, col=col2track)

In [14]:
res.groupby(['agency_id_new','agency_name_new']).route_id.count().sort_values(ascending=False).to_frame()

,,route_id
agency_id_new,agency_name_new,
3.0,אגד,108
18.0,קווים,86
5.0,דן,62
2.0,רכבת ישראל,49
7.0,נסיעות ותיירות,26
15.0,מטרופולין,14
4.0,אגד תעבורה,14
16.0,סופרבוס,8
14.0,נתיב אקספרס,7


In [15]:
res.head(3)

,route_id,added,deleted,agency_id_old,agency_name_old,route_short_name_old,route_long_name_old,start_stop_city_old,end_stop_city_old,date_old,all_start_time_old,agency_id_new,agency_name_new,route_short_name_new,route_long_name_new,start_stop_city_new,end_stop_city_new,date_new,all_start_time_new
0,345,23:10:00,23:00:00,14.0,נתיב אקספרס,40,ת. מרכזית נהריה/רציפים-נהריה<->אבירי...,נהריה,אבירים,2019-11-30,21:35:00;23:00:00,14.0,נתיב אקספרס,40,ת. מרכזית נהריה/רציפים-נהריה<->אבירי...,נהריה,אבירים,2020-01-11,21:35:00;23:10:00
1,1099,NaN,21:50:00;21:20:00;19:50:00;19:20:00;...,25.0,אפיקים,23,בן ציון גליס/צבי ברגמן-פתח תקווה<->ב...,פתח תקווה,פתח תקווה,2019-11-30,19:20:00;19:50:00;20:20:00;20:50:00;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1100,NaN,21:25:00;21:00:00;23:00:00;19:30:00;...,25.0,אפיקים,23,בית חולים בית רבקה-פתח תקווה<->בן צי...,פתח תקווה,פתח תקווה,2019-11-30,18:30:00;19:00:00;19:30:00;20:00:00;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## add deleted by hours cols

for monitoring changes in all_start_time column in specific time slot

In [16]:
def set_deleted_between(min_val, max_val):
    deleted_between = res.apply(
        lambda r: ";".join([h for h in r.deleted.split(';') if int('1'+h.replace(':','').strip()) >= int('1'+min_val)\
                                                        and int('1'+h.replace(':','').strip()) <= int('1'+max_val)])\
            if r.deleted is not np.nan else np.nan,
            axis=1)

    return deleted_between.mask(deleted_between.astype(str)=='') 

In [17]:
### set time slot (min_hour & max_hour)
min_hour, max_hour = '000000', '160000'

### run set_deleted_between function

col_name = f'deleted_between_{min_hour[:4]}_{max_hour[:4]}'

res[col_name] = set_deleted_between(min_hour, max_hour)

res[f'{col_name}_cnt'] = res.apply(lambda r: len(r[col_name].split(';')) if type(r[col_name]) == str else 0,
                                   axis=1)

display(res[res[col_name].notnull()].shape)


res[res[col_name].notnull()].head()

(29, 21)

,route_id,added,deleted,agency_id_old,agency_name_old,route_short_name_old,route_long_name_old,start_stop_city_old,end_stop_city_old,date_old,all_start_time_old,agency_id_new,agency_name_new,route_short_name_new,route_long_name_new,start_stop_city_new,end_stop_city_new,date_new,all_start_time_new,deleted_between_0000_1600,deleted_between_0000_1600_cnt
9,1415,17:15:00;09:45:00;15:00:00;11:00:00;...,07:15:00,7.0,נסיעות ותיירות,28,סחנין יציאה מערבית-סח'נין<->תחנה מרכ...,סח'נין,נצרת,2019-11-30,06:00:00;07:15:00;08:00:00;08:35:00,7.0,נסיעות ותיירות,28,סחנין יציאה מערבית-סח'נין<->תחנה מרכ...,סח'נין,נצרת,2020-01-11,06:00:00;07:00:00;08:00:00;08:35:00;...,07:15:00,1
17,1482,13:35:00,12:30:00,7.0,נסיעות ותיירות,52,כאוכב ג'/ככר-כאוכב אבו אל היגא<->מכל...,כאוכב אבו אל היגא,אעבלין,2019-11-30,07:10:00;12:30:00,7.0,נסיעות ותיירות,52,כאוכב ג'/ככר-כאוכב אבו אל היגא<->מכל...,כאוכב אבו אל היגא,אעבלין,2020-01-11,07:10:00;13:35:00,12:30:00,1
18,1485,14:40:00,13:40:00,7.0,נסיעות ותיירות,52,מכללת מר אליאס-אעבלין<->כאוכב ג'/ככר...,אעבלין,כאוכב אבו אל היגא,2019-11-30,08:20:00;13:40:00,7.0,נסיעות ותיירות,52,מכללת מר אליאס-אעבלין<->כאוכב ג'/ככר...,אעבלין,כאוכב אבו אל היגא,2020-01-11,08:20:00;14:40:00,13:40:00,1
19,1523,13:31:00;13:30:00,12:30:00;12:31:00,7.0,נסיעות ותיירות,59,צומת דיר חנא-דייר חנא<->מכללת מר אלי...,דייר חנא,אעבלין,2019-11-30,07:10:00;07:11:00;12:30:00;12:31:00,7.0,נסיעות ותיירות,59,צומת דיר חנא-דייר חנא<->מכללת מר אלי...,דייר חנא,אעבלין,2020-01-11,07:10:00;07:11:00;13:30:00;13:31:00,12:30:00;12:31:00,2
20,1526,14:41:00;14:40:00,13:40:00;13:41:00,7.0,נסיעות ותיירות,59,מכללת מר אליאס-אעבלין<->דייר חנא ו'/...,אעבלין,דייר חנא,2019-11-30,08:30:00;08:31:00;13:40:00;13:41:00,7.0,נסיעות ותיירות,59,מכללת מר אליאס-אעבלין<->דייר חנא ו'/...,אעבלין,דייר חנא,2020-01-11,08:30:00;08:31:00;14:40:00;14:41:00,13:40:00;13:41:00,2


In [18]:
### set time slot (min_hour & max_hour)
min_hour, max_hour = '180100', '235900'

### run set_deleted_between function

col_name = f'deleted_between_{min_hour[:4]}_{max_hour[:4]}'

res[col_name] = set_deleted_between(min_hour, max_hour)

res[f'{col_name}_cnt'] = res.apply(lambda r: len(r[col_name].split(';')) if type(r[col_name]) == str else 0,
                                   axis=1)

display(res[res[col_name].notnull()].shape)


res[res[col_name].notnull()].head()

(268, 23)

,route_id,added,deleted,agency_id_old,agency_name_old,route_short_name_old,route_long_name_old,start_stop_city_old,end_stop_city_old,date_old,all_start_time_old,agency_id_new,agency_name_new,route_short_name_new,route_long_name_new,start_stop_city_new,end_stop_city_new,date_new,all_start_time_new,deleted_between_0000_1600,deleted_between_0000_1600_cnt,deleted_between_1801_2359,deleted_between_1801_2359_cnt
0,345,23:10:00,23:00:00,14.0,נתיב אקספרס,40,ת. מרכזית נהריה/רציפים-נהריה<->אבירי...,נהריה,אבירים,2019-11-30,21:35:00;23:00:00,14.0,נתיב אקספרס,40,ת. מרכזית נהריה/רציפים-נהריה<->אבירי...,נהריה,אבירים,2020-01-11,21:35:00;23:10:00,NaN,0,23:00:00,1
1,1099,NaN,21:50:00;21:20:00;19:50:00;19:20:00;...,25.0,אפיקים,23,בן ציון גליס/צבי ברגמן-פתח תקווה<->ב...,פתח תקווה,פתח תקווה,2019-11-30,19:20:00;19:50:00;20:20:00;20:50:00;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21:50:00;21:20:00;19:50:00;19:20:00;...,10
2,1100,NaN,21:25:00;21:00:00;23:00:00;19:30:00;...,25.0,אפיקים,23,בית חולים בית רבקה-פתח תקווה<->בן צי...,פתח תקווה,פתח תקווה,2019-11-30,18:30:00;19:00:00;19:30:00;20:00:00;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21:25:00;21:00:00;23:00:00;19:30:00;...,11
21,2240,NaN,18:25:00,5.0,דן,2,קניון איילון-רמת גן<->אהרונוביץ/השלו...,רמת גן,בני ברק,2019-11-30,18:25:00;18:55:00;19:30:00;20:00:00;...,5.0,דן,2,קניון איילון-רמת גן<->אהרונוביץ/השלו...,רמת גן,בני ברק,2020-01-11,18:55:00;19:30:00;20:00:00;20:30:00;...,NaN,0,18:25:00,1
22,2242,NaN,18:25:00,5.0,דן,2,אהרונוביץ/השלושה-בני ברק<->קניון איי...,בני ברק,רמת גן,2019-11-30,18:25:00;18:55:00;19:25:00;19:55:00;...,5.0,דן,2,אהרונוביץ/השלושה-בני ברק<->קניון איי...,בני ברק,רמת גן,2020-01-11,18:55:00;19:25:00;19:55:00;20:30:00;...,NaN,0,18:25:00,1


In [19]:
### set time slot (min_hour & max_hour)
min_hour, max_hour = '160100', '180000'

### run set_deleted_between function

col_name = f'deleted_between_{min_hour[:4]}_{max_hour[:4]}'

res[col_name] = set_deleted_between(min_hour, max_hour)

res[f'{col_name}_cnt'] = res.apply(lambda r: len(r[col_name].split(';')) if type(r[col_name]) == str else 0,
                                   axis=1)

display(res[res[col_name].notnull()].shape)


res[res[col_name].notnull()].head()

(147, 25)

,route_id,added,deleted,agency_id_old,agency_name_old,route_short_name_old,route_long_name_old,start_stop_city_old,end_stop_city_old,date_old,all_start_time_old,agency_id_new,agency_name_new,route_short_name_new,route_long_name_new,start_stop_city_new,end_stop_city_new,date_new,all_start_time_new,deleted_between_0000_1600,deleted_between_0000_1600_cnt,deleted_between_1801_2359,deleted_between_1801_2359_cnt,deleted_between_1601_1800,deleted_between_1601_1800_cnt
23,2259,NaN,17:48:00,5.0,דן,5,ת.מרכזית ת''א ק.4/רציפים-תל אביב יפו...,תל אביב יפו,תל אביב יפו,2019-11-30,17:48:00;18:04:00;18:20:00;18:36:00;...,5.0,דן,5,ת.מרכזית ת''א ק.4/רציפים-תל אביב יפו...,תל אביב יפו,תל אביב יפו,2020-01-11,18:04:00;18:20:00;18:36:00;18:52:00;...,NaN,0,NaN,0,17:48:00,1
24,2261,17:58:00,17:56:00;17:40:00,5.0,דן,5,ת. רכבת ת''א מרכז/על פרשת דרכים-תל א...,תל אביב יפו,תל אביב יפו,2019-11-30,17:40:00;17:56:00;18:12:00;18:28:00;...,5.0,דן,5,ת. רכבת ת''א מרכז/על פרשת דרכים-תל א...,תל אביב יפו,תל אביב יפו,2020-01-11,17:58:00;18:12:00;18:28:00;18:44:00;...,NaN,0,NaN,0,17:56:00;17:40:00,2
25,2293,17:10:00;19:35:00;22:40:00;21:25:00;...,21:05:00;21:40:00;18:35:00;19:50:00;...,5.0,דן,9,ח.רידינג-תל אביב יפו<->מסוף הטייסים-...,תל אביב יפו,תל אביב יפו,2019-11-30,17:25:00;18:00:00;18:35:00;19:15:00;...,5.0,דן,9,ח.רידינג-תל אביב יפו<->מסוף הטייסים-...,תל אביב יפו,תל אביב יפו,2020-01-11,17:10:00;17:45:00;18:20:00;19:00:00;...,NaN,0,21:05:00;21:40:00;18:35:00;19:50:00;...,9,17:25:00;18:00:00,2
26,2294,23:35:00;21:05:00;18:40:00;22:20:00;...,21:20:00;18:55:00;18:20:00;20:45:00;...,5.0,דן,9,מסוף הטייסים-תל אביב יפו<->רדינג-תל ...,תל אביב יפו,תל אביב יפו,2019-11-30,17:40:00;18:20:00;18:55:00;19:30:00;...,5.0,דן,9,מסוף הטייסים-תל אביב יפו<->רדינג-תל ...,תל אביב יפו,תל אביב יפו,2020-01-11,17:25:00;18:05:00;18:40:00;19:15:00;...,NaN,0,21:20:00;18:55:00;18:20:00;20:45:00;...,10,17:40:00,1
27,2296,NaN,17:40:00,5.0,דן,10,ת. רכבת ת''א מרכז-תל אביב יפו<->מתחם...,תל אביב יפו,ראשון לציון,2019-11-30,17:40:00;18:15:00;19:00:00;19:45:00;...,5.0,דן,10,ת. רכבת ת''א מרכז-תל אביב יפו<->מתחם...,תל אביב יפו,ראשון לציון,2020-01-11,18:15:00;19:00:00;19:45:00;20:30:00;...,NaN,0,NaN,0,17:40:00,1


# Examples

In [20]:
route_short_names = ['25','27','31','34','44','45','56','61','64','66','353','383','446','487','940','959']

In [21]:
res[res.route_short_name_new.fillna(res.route_short_name_old).str.strip().isin(route_short_names)]\
    .sort_values(by = ['route_short_name_new', 'route_short_name_old']).head()

,route_id,added,deleted,agency_id_old,agency_name_old,route_short_name_old,route_long_name_old,start_stop_city_old,end_stop_city_old,date_old,all_start_time_old,agency_id_new,agency_name_new,route_short_name_new,route_long_name_new,start_stop_city_new,end_stop_city_new,date_new,all_start_time_new,deleted_between_0000_1600,deleted_between_0000_1600_cnt,deleted_between_1801_2359,deleted_between_1801_2359_cnt,deleted_between_1601_1800,deleted_between_1601_1800_cnt
41,2374,NaN,17:40:00,5.0,דן,25,ת. רכבת אוניברסיטה/מערב-תל אביב יפו<...,תל אביב יפו,בת ים,2019-11-30,17:40:00;18:15:00;18:50:00;19:25:00;...,5.0,דן,25,ת. רכבת אוניברסיטה/מערב-תל אביב יפו<...,תל אביב יפו,בת ים,2020-01-11,18:15:00;18:50:00;19:25:00;19:55:00;...,NaN,0,NaN,0,17:40:00,1
73,5530,NaN,18:00:00,3.0,אגד,25,מסוף אגד נווה יעקב מזרח/הרב וינוגרד-...,ירושלים,ירושלים,2019-11-30,18:00:00;18:15:00;18:30:00;18:45:00;...,3.0,אגד,25,מסוף אגד נווה יעקב מזרח/הרב וינוגרד-...,ירושלים,ירושלים,2020-01-11,18:15:00;18:30:00;18:45:00;19:00:00;...,NaN,0,NaN,0,18:00:00,1
134,10277,20:25:00;19:40:00,21:15:00;22:15:00,4.0,אגד תעבורה,25,בית גיל הזהב/השקד-נתניה<->ת. רכבת נת...,נתניה,נתניה,2019-11-30,20:55:00;21:15:00;21:55:00;22:15:00,4.0,אגד תעבורה,25,בית גיל הזהב/השקד-נתניה<->ת. רכבת נת...,נתניה,נתניה,2020-01-11,19:40:00;20:25:00;20:55:00;21:55:00,NaN,0,21:15:00;22:15:00,2,NaN,0
135,10278,20:40:00;22:15:00,22:10:00;21:40:00,4.0,אגד תעבורה,25,ת. רכבת נתניה-נתניה<->בית גיל הזהב/ה...,נתניה,נתניה,2019-11-30,21:10:00;21:40:00;22:10:00;23:00:00,4.0,אגד תעבורה,25,ת. רכבת נתניה-נתניה<->בית גיל הזהב/ה...,נתניה,נתניה,2020-01-11,20:40:00;21:10:00;22:15:00;23:00:00,NaN,0,22:10:00;21:40:00,2,NaN,0
125,10207,NaN,18:00:00,3.0,אגד,27,כניסה ראשית/הדסה עין כרם-ירושלים<->כ...,ירושלים,ירושלים,2019-11-30,18:00:00;18:20:00;18:40:00;19:00:00;...,3.0,אגד,27,כניסה ראשית/הדסה עין כרם-ירושלים<->כ...,ירושלים,ירושלים,2020-01-11,18:20:00;18:40:00;19:00:00;19:20:00;...,NaN,0,NaN,0,18:00:00,1
